In [2]:
import os
import random
import skimage.io
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
from skimage import color
from skimage import filters

import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras import backend as K

# Allow image embeding in notebook
%matplotlib inline

In [3]:
import cv2 as cv2
import tensorflow as tf

In [4]:
def load_data(data_dir):
    """Loads a data set and returns two lists:
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    This function could be used for both training data and testing data.
    """
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        
        label_dir = os.path.join(data_dir, d)
        # print(label_dir) yes
        file_names = [os.path.join(label_dir, f) 
                      for f in os.listdir(label_dir) if f.endswith(".ppm")]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        # print(file_names)
        for f in file_names:
            print(f)
            img = skimage.io.imread(f)
            images.append(img)
            print(img.shape)
            labels.append(int(d))
    return images, labels

In [5]:
from sklearn.model_selection import train_test_split

Load & Parse Data

In [ ]:
ROOT_PATH = "insert directory of your data"
train_data_dir = os.path.join(ROOT_PATH,"insert directory of your training data")
test_data_dir =  os.path.join(ROOT_PATH,"insert directory of your testing data")
images, labels = load_data(train_data_dir)
test_images, test_labels = load_data(test_data_dir)

Resizing Training  data

In [7]:
images32 = [skimage.transform.resize(image, (32, 32), mode='constant')
                for image in images]

In [9]:
test_images32 = [skimage.transform.resize(img, (32, 32), mode='constant')
                for img in test_images]

In [10]:
image_data = np.array(images32)
#image_labels = np.array(image_labels)
print(image_data.shape)
label_data = np.array(labels)
print(label_data.shape)

(25, 32, 32, 3)
(25,)


In [11]:
test_image_data = np.array(test_images32)
#image_labels = np.array(image_labels)
print(test_image_data.shape)
test_label_data = np.array(test_labels)
print(test_label_data.shape)

(104, 32, 32, 3)
(104,)


In [12]:
X_train, X_val, y_train,y_val = train_test_split(image_data, label_data,test_size=0.3, random_state=0,shuffle=True)


print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)


(17, 32, 32, 3)
(8, 32, 32, 3)
(17,)
(8,)


In [13]:
y_train = keras.utils.to_categorical(y_train,7)
y_val = keras.utils.to_categorical(y_val,7)


print(y_train.shape)
print(y_val.shape)

(17, 7)
(8, 7)


In [14]:
X_test, X_valid, y_test, y_valid=train_test_split(test_image_data, test_label_data, random_state=0,shuffle=True)

In [15]:
print(X_test.shape)
print(X_valid.shape)
print(y_test.shape)
print(y_valid.shape)

(78, 32, 32, 3)
(26, 32, 32, 3)
(78,)
(26,)


In [16]:
y_test = keras.utils.to_categorical(y_test,7)
y_valid = keras.utils.to_categorical(y_valid,7)


print(y_test.shape)
print(y_valid.shape)

(78, 7)
(26, 7)


Expand Training data

In [17]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.data import AUTOTUNE
from imutils import paths
import matplotlib.pyplot as plt
import tensorflow as tf
import argparse
import os
from tensorflow.keras.preprocessing.image  import ImageDataGenerator

In [18]:
data_augmentation  = ImageDataGenerator(
		                 rotation_range = 20,
										 zoom_range = 0.15,
										 width_shift_range = 0.1,
										 horizontal_flip = True,
										 vertical_flip = True,
                     featurewise_center=True,
										 fill_mode ="nearest")

NN layer

In [19]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters =32, kernel_size=(3,3),activation='relu',input_shape =X_train.shape[1:]),
    keras.layers.Conv2D(filters =32, kernel_size=(3,3),activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Dropout(rate=0.25),

    keras.layers.Conv2D(filters =64, kernel_size=(3,3),activation='relu'),
    keras.layers.Conv2D(filters =64, kernel_size=(3,3),activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Dropout(rate=0.25),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation ='relu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Dropout(rate=0.5),

    keras.layers.Dense(7, activation ='softmax')

])

In [20]:
epochs = 100
lr =0.001

optimizer = keras.optimizers.Adam(learning_rate=lr, decay = lr/(epochs*0.5))
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Training the network for 100 epochs

In [21]:
history = model.fit(data_augmentation.flow(X_train,y_train, batch_size=20),epochs=epochs, validation_data=(X_val, y_val))

/usr/local/lib/python3.8/dist-packages/keras/preprocessing/image.py:1663: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 3.1874 - accuracy: 0.0588 - val_loss: 1.9338 - val_accuracy: 0.2500
Epoch 2/100
1/1 [==============================] - 0s 139ms/step - loss: 2.9985 - accuracy: 0.1176 - val_loss: 1.9090 - val_accuracy: 0.3750
Epoch 3/100
1/1 [==============================] - 0s 110ms/step - loss: 2.5109 - accuracy: 0.1765 - val_loss: 1.8796 - val_accuracy: 0.2500
Epoch 4/100
1/1 [==============================] - 0s 127ms/step - loss: 2.3294 - accuracy: 0.1765 - val_loss: 1.8480 - val_accuracy: 0.2500
Epoch 5/100
1/1 [==============================] - 0s 135ms/step - loss: 2.2860 - accuracy: 0.1176 - val_loss: 1.8290 - val_accuracy: 0.2500
Epoch 6/100
1/1 [==============================] - 0s 120ms/step - loss: 1.7971 - accuracy: 0.2353 - val_loss: 1.8123 - val_accuracy: 0.2500
Epoch 7/100
1/1 [==============================] - 0s 116ms/step - loss: 1.7112 - accuracy: 0.4118 - val_loss: 1.7986 - val_accuracy: 0.2500
Epoch 8/100
1/1 

In [24]:
model.evaluate(x=X_test, y=y_test)

3/3 [==============================] - 0s 22ms/step - loss: 0.9489 - accuracy: 0.9103


[0.9489033818244934, 0.9102563858032227]